# FIFA Players Classification

This notebook contains the code to train models for the FIFA Classification problem based on their attributes

## Step 1: Import libraries

Libraries we will be using:
- **tensorflow** and **keras**: APIs for neural network model construction and training
- **numpy**: Numerical python library.
- **pandas**: Will be used to work with dataframes from .csv files.
- **sklearn**: Will be used to modify the labels of the data and do some statistical modifications.
- **matplotlib**: Will be used to plot graphics.
- **time**: To measure training time.
- **datetime**: To get the datetime a new model is created and use that to create its name

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import datetime

## Step 2: Load data

We first define the path to our data

In [2]:
DATA_FOLDER = "../Data"

ATT_FILE = f"{DATA_FOLDER}/FootballPlayerPreparedCleanAttributes.csv"
IMPUTED_ATT_FILE = f"{DATA_FOLDER}/ImputedFootballPlayerPreparedCleanAttributes.csv"

ONE_HOT_LABEL_FILE = f"{DATA_FOLDER}/FootballPlayerOneHotEncodedClasses.csv"
IMPUTED_ONE_HOT_LABEL_FILE = f"{DATA_FOLDER}/ImputedFootballPlayerPreparedCleanAttributes.csv"

Specify training options:
- **IMPUTED_DATA**: We will use file with data imputed
- **TRAIN_RATE**: The amount of instances to use during training, it will also determine the dev and test instances number indirectly

In [3]:
IMPUTED_DATA = False    # Use imputed dataset
TRAIN_RATE = 0.8        # The division of the data for training, validation and test

Because our data has more than 10000, we will be using 80-10-10 split in our dataset:
- **80%** for training
- **10%** for validation/dev test
- **10%** for final test

In [4]:
if IMPUTED_DATA:
    attributes = pd.read_csv(ATT_IMPUTED_FILE)
    labels = pd.read_csv(ONE_HOT_LABEL_IMPUTED_FILE)
else:
    attributes = pd.read_csv(ATT_FILE)
    labels = pd.read_csv(ONE_HOT_LABEL_FILE)

n_total_instances = attributes.shape[0]
n_train_instances = int(n_total_instances * TRAIN_RATE)
n_dev_instances = int((n_total_instances - n_train_instances)/2)

instances_train = attributes.values[:n_train_instances]
labels_train = labels.values[:n_train_instances]

instances_dev = attributes.values[n_train_instances: n_train_instances + n_dev_instances]
labels_dev = labels.values[n_train_instances: n_train_instances + n_dev_instances]

print("instances_train:", instances_train.shape)
print("labels_train:", labels_train.shape)
print("instances_dev:", instances_dev.shape)
print("labels_dev:",labels_dev.shape)

instances_train: (12897, 17)
labels_train: (12897, 4)
instances_dev: (1612, 17)
labels_dev: (1612, 4)


## Step 3: Initialize variables
- **INPUT_SIZE**: The size of the training instances
- **OUTPUT_SIZE**: The number of possible classes
- **NUM_TRAINING_EXAMPLES**: The number of possible classes
- **NUM_DEV_EXAMPLES**: The number of validation/dev test instances

In [5]:
INPUT_SIZE = instances_train.shape[1]
OUTPUT_SIZE = labels_train.shape[1]
NUM_TRAINING_EXAMPLES = int(round(instances_train.shape[0]/1))
NUM_DEV_EXAMPLES = int(round(instances_dev.shape[0]/1))

## Step 4: Set hyperparameters

- **LEARNING_RATE** $\alpha$: The step size in the learning process
- **BATCH_SIZE**: The size of each batch
- **N_HIDDEN**: Number of neurons in the hidden.

The learning process stops when it reaches 20,000 learning iterations. An iteration comprises presenting the network *batch_size* learning examples and then adjusting the weights (parameters).

In [6]:
N_EPOCHS = 400
LEARNING_RATE = 0.1
BATCH_SIZE = 256
ACTIVATION = "elu"
N_HIDDEN = [512, 128, 32, 128, 512]
REGULARIZER = keras.regularizers.l1(0.0001)
INITIALIZER = keras.initializers.he_normal(seed=None)
LOSS = tf.keras.losses.categorical_crossentropy
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE)
METRICS = ["categorical_accuracy"]

## Step 5: Build the model

In [7]:
def architecture1():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,), batch_size=None))
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons, activation=ACTIVATION))
    model.add(keras.layers.Dense(OUTPUT_SIZE, activation="softmax"))
    
    model_name = "D512"
    return model, model_name


def architecture2():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons,
                                     activation=ACTIVATION,
                                     kernel_regularizer=REGULARIZER
                                    ))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE, 
                                 activation="softmax"))
    
    model_name = "D128_D64_D32"
    return model, model_name

def architecture3():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation(ACTIVATION))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE,
                                 activation="softmax"
                                ))
    model_name = "D128_BN_D64_BN_D32_BN"
    return model, model_name

def architecture4():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons,
                                     kernel_regularizer=REGULARIZER,
                                     kernel_initializer=keras.initializers.HeNormal(seed=None),
                                     use_bias=False
                                     ))
    
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation(ACTIVATION))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE,
                                 activation="softmax"
                                ))
    model_name = "D2048_BN_D1024_BN_D512_BN_D256_BN_D128_BN_D64_BN_D32_BN"
    return model, model_name


def architecture5():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons,
                                     kernel_regularizer=REGULARIZER,
                                     kernel_initializer=keras.initializers.HeNormal(seed=None),
                                     use_bias=False
                                     ))
    
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation(ACTIVATION))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE,
                                 activation="softmax"
                                ))
    
    model_name = "D512_BN_D128_BN_D32_BN_D128_BN_D512_BN"
    return model, model_name

def architecture5dr():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons,
                                     kernel_regularizer=REGULARIZER,
                                     kernel_initializer=keras.initializers.HeNormal(seed=None),
                                     use_bias=False
                                     ))
        model.add(keras.layers.Dropout(rate=0.1))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation(ACTIVATION))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE,
                                 activation="softmax"
                                ))
    
    model_name = "D512_DR_BN_D128_DR_BN_D32_DR_BN_D128_DR_BN_D512_DR_BN"
    return model, model_name


def compile_model(model):
    my_model.compile(loss=LOSS,
                     optimizer=OPTIMIZER,
                     metrics=METRICS)

In [8]:
my_model, model_name = architecture5dr()

In [9]:
compile_model(my_model)

## Step 6: Train the model

In [ ]:
start_time = time.perf_counter()
history = my_model.fit(instances_train,
                       labels_train,
                       batch_size=BATCH_SIZE,
                       epochs=N_EPOCHS,
                       verbose=1,
                       validation_data=(instances_dev, labels_dev))
end_time = time.perf_counter()

Epoch 1/400
51/51 [==============================] - 3s 17ms/step - loss: 2.1424 - categorical_accuracy: 0.6526 - val_loss: 2.8992 - val_categorical_accuracy: 0.5310
Epoch 2/400
51/51 [==============================] - 1s 13ms/step - loss: 1.9852 - categorical_accuracy: 0.7226 - val_loss: 2.4279 - val_categorical_accuracy: 0.6098
Epoch 3/400
51/51 [==============================] - 1s 13ms/step - loss: 1.9497 - categorical_accuracy: 0.7368 - val_loss: 1.9375 - val_categorical_accuracy: 0.7500
Epoch 4/400
51/51 [==============================] - 1s 12ms/step - loss: 1.9320 - categorical_accuracy: 0.7431 - val_loss: 2.0408 - val_categorical_accuracy: 0.7091
Epoch 5/400
51/51 [==============================] - 1s 12ms/step - loss: 1.9112 - categorical_accuracy: 0.7473 - val_loss: 1.9310 - val_categorical_accuracy: 0.7457
Epoch 6/400
51/51 [==============================] - 1s 12ms/step - loss: 1.9091 - categorical_accuracy: 0.7469 - val_loss: 1.8789 - val_categorical_accuracy: 0.7438
Epoc

51/51 [==============================] - 1s 18ms/step - loss: 1.5417 - categorical_accuracy: 0.7831 - val_loss: 1.5008 - val_categorical_accuracy: 0.7934
Epoch 51/400
51/51 [==============================] - 1s 17ms/step - loss: 1.5311 - categorical_accuracy: 0.7821 - val_loss: 1.5360 - val_categorical_accuracy: 0.7767
Epoch 52/400
51/51 [==============================] - 1s 18ms/step - loss: 1.5338 - categorical_accuracy: 0.7795 - val_loss: 1.5055 - val_categorical_accuracy: 0.7903
Epoch 53/400
51/51 [==============================] - 1s 17ms/step - loss: 1.5133 - categorical_accuracy: 0.7883 - val_loss: 1.5093 - val_categorical_accuracy: 0.7866
Epoch 54/400
51/51 [==============================] - 1s 18ms/step - loss: 1.5036 - categorical_accuracy: 0.7913 - val_loss: 1.4857 - val_categorical_accuracy: 0.7934
Epoch 55/400
51/51 [==============================] - 1s 17ms/step - loss: 1.5048 - categorical_accuracy: 0.7879 - val_loss: 1.4995 - val_categorical_accuracy: 0.7761
Epoch 56/40

51/51 [==============================] - 1s 18ms/step - loss: 1.2671 - categorical_accuracy: 0.7927 - val_loss: 1.2604 - val_categorical_accuracy: 0.7947
Epoch 100/400
51/51 [==============================] - 1s 19ms/step - loss: 1.2710 - categorical_accuracy: 0.7903 - val_loss: 1.2296 - val_categorical_accuracy: 0.8046
Epoch 101/400
51/51 [==============================] - 1s 17ms/step - loss: 1.2690 - categorical_accuracy: 0.7856 - val_loss: 1.2403 - val_categorical_accuracy: 0.7984
Epoch 102/400
51/51 [==============================] - 1s 18ms/step - loss: 1.2657 - categorical_accuracy: 0.7868 - val_loss: 1.2325 - val_categorical_accuracy: 0.8065
Epoch 103/400
51/51 [==============================] - 1s 17ms/step - loss: 1.2521 - categorical_accuracy: 0.7933 - val_loss: 1.2783 - val_categorical_accuracy: 0.7717
Epoch 104/400
51/51 [==============================] - 1s 18ms/step - loss: 1.2461 - categorical_accuracy: 0.7909 - val_loss: 1.2239 - val_categorical_accuracy: 0.7984
Epoch 

Epoch 148/400
51/51 [==============================] - 1s 19ms/step - loss: 1.0910 - categorical_accuracy: 0.7868 - val_loss: 1.0966 - val_categorical_accuracy: 0.7785
Epoch 149/400
51/51 [==============================] - 1s 18ms/step - loss: 1.0771 - categorical_accuracy: 0.7975 - val_loss: 1.0959 - val_categorical_accuracy: 0.7854
Epoch 150/400
51/51 [==============================] - 1s 18ms/step - loss: 1.0821 - categorical_accuracy: 0.7910 - val_loss: 1.2074 - val_categorical_accuracy: 0.7109
Epoch 151/400
51/51 [==============================] - 1s 18ms/step - loss: 1.0726 - categorical_accuracy: 0.7920 - val_loss: 1.1058 - val_categorical_accuracy: 0.7686
Epoch 152/400
51/51 [==============================] - 1s 19ms/step - loss: 1.0684 - categorical_accuracy: 0.7952 - val_loss: 1.3383 - val_categorical_accuracy: 0.6873
Epoch 153/400
51/51 [==============================] - 1s 19ms/step - loss: 1.0760 - categorical_accuracy: 0.7884 - val_loss: 1.0432 - val_categorical_accuracy:

Epoch 197/400
51/51 [==============================] - 1s 17ms/step - loss: 0.9494 - categorical_accuracy: 0.7934 - val_loss: 1.0993 - val_categorical_accuracy: 0.7413
Epoch 198/400
51/51 [==============================] - 1s 18ms/step - loss: 0.9416 - categorical_accuracy: 0.7945 - val_loss: 0.9814 - val_categorical_accuracy: 0.7730
Epoch 199/400
51/51 [==============================] - 1s 17ms/step - loss: 0.9518 - categorical_accuracy: 0.7909 - val_loss: 0.9387 - val_categorical_accuracy: 0.7854
Epoch 200/400
51/51 [==============================] - 1s 17ms/step - loss: 0.9412 - categorical_accuracy: 0.7893 - val_loss: 0.9715 - val_categorical_accuracy: 0.7736
Epoch 201/400
51/51 [==============================] - 1s 18ms/step - loss: 0.9451 - categorical_accuracy: 0.7940 - val_loss: 0.9417 - val_categorical_accuracy: 0.7872
Epoch 202/400
51/51 [==============================] - 1s 17ms/step - loss: 0.9416 - categorical_accuracy: 0.7924 - val_loss: 0.9814 - val_categorical_accuracy:

Epoch 246/400
51/51 [==============================] - 1s 18ms/step - loss: 0.8408 - categorical_accuracy: 0.8003 - val_loss: 0.8944 - val_categorical_accuracy: 0.7785
Epoch 247/400
51/51 [==============================] - 1s 18ms/step - loss: 0.8536 - categorical_accuracy: 0.7917 - val_loss: 0.8578 - val_categorical_accuracy: 0.7792
Epoch 248/400
51/51 [==============================] - 1s 18ms/step - loss: 0.8410 - categorical_accuracy: 0.7980 - val_loss: 0.9450 - val_categorical_accuracy: 0.7550
Epoch 249/400
51/51 [==============================] - 1s 18ms/step - loss: 0.8484 - categorical_accuracy: 0.7928 - val_loss: 0.9458 - val_categorical_accuracy: 0.7395
Epoch 250/400
51/51 [==============================] - 1s 18ms/step - loss: 0.8328 - categorical_accuracy: 0.8014 - val_loss: 0.8584 - val_categorical_accuracy: 0.7779
Epoch 251/400
51/51 [==============================] - 1s 17ms/step - loss: 0.8478 - categorical_accuracy: 0.7945 - val_loss: 0.8376 - val_categorical_accuracy:

Epoch 295/400
51/51 [==============================] - 1s 17ms/step - loss: 0.7795 - categorical_accuracy: 0.7940 - val_loss: 0.8294 - val_categorical_accuracy: 0.7630
Epoch 296/400
51/51 [==============================] - 1s 18ms/step - loss: 0.7816 - categorical_accuracy: 0.7979 - val_loss: 0.7688 - val_categorical_accuracy: 0.7928
Epoch 297/400
51/51 [==============================] - 1s 17ms/step - loss: 0.7730 - categorical_accuracy: 0.7961 - val_loss: 0.7797 - val_categorical_accuracy: 0.7854
Epoch 298/400
51/51 [==============================] - 1s 17ms/step - loss: 0.7718 - categorical_accuracy: 0.7939 - val_loss: 1.0148 - val_categorical_accuracy: 0.6842
Epoch 299/400
51/51 [==============================] - 1s 18ms/step - loss: 0.7659 - categorical_accuracy: 0.7950 - val_loss: 0.7414 - val_categorical_accuracy: 0.8027
Epoch 300/400
51/51 [==============================] - 1s 17ms/step - loss: 0.7695 - categorical_accuracy: 0.7964 - val_loss: 0.7868 - val_categorical_accuracy:

Epoch 344/400
51/51 [==============================] - 1s 17ms/step - loss: 0.7240 - categorical_accuracy: 0.7935 - val_loss: 0.7384 - val_categorical_accuracy: 0.7773
Epoch 345/400
51/51 [==============================] - 1s 18ms/step - loss: 0.7304 - categorical_accuracy: 0.7977 - val_loss: 0.7187 - val_categorical_accuracy: 0.7947
Epoch 346/400
51/51 [==============================] - 1s 17ms/step - loss: 0.7134 - categorical_accuracy: 0.7998 - val_loss: 0.9574 - val_categorical_accuracy: 0.6842
Epoch 347/400
51/51 [==============================] - 1s 18ms/step - loss: 0.7230 - categorical_accuracy: 0.7941 - val_loss: 0.9652 - val_categorical_accuracy: 0.6818
Epoch 348/400
51/51 [==============================] - 1s 17ms/step - loss: 0.7184 - categorical_accuracy: 0.7948 - val_loss: 0.7558 - val_categorical_accuracy: 0.7736
Epoch 349/400
51/51 [==============================] - 1s 18ms/step - loss: 0.7074 - categorical_accuracy: 0.7996 - val_loss: 0.7367 - val_categorical_accuracy:

## Step 7: Analyze results

In [ ]:
results=pd.DataFrame(history.history)
results.plot(figsize=(8, 5))
plt.grid(True)
plt.xlabel ("Epochs")
plt.ylabel ("Accuracy - Mean Log Loss")
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
training_accuracy = round(results.categorical_accuracy.values[-1:][0]*100, 2)
val_accuracy = round(results.val_categorical_accuracy.values[-1:][0]*100, 2)
training_error = round((1 - results.categorical_accuracy.values[-1:][0])*100, 2)
test_error = round((1 - results.val_categorical_accuracy.values[-1:][0])*100, 2)
human_error = 5
bias = round(training_error - human_error, 2)
variance = round(test_error - training_error, 2)
training_time = round(end_time - start_time)
print("Training accuracy: ", training_accuracy, "%")
print("Validationa accuracy", val_accuracy, "%")
print("Error (training):", training_error, "%")
print("Error (test):", test_error, "%")
print("Bias:", bias)
print("Variance:", variance)
print("Time: ", training_time, "seconds")

In [ ]:
def draw_confusion_matrix(cm, categories, normalize=True):
    # Draw confusion matrix
    fig = plt.figure(figsize=[3*pow(len(categories), 0.5), 2*pow(len(categories), 0.5)])
    ax = fig.add_subplot(111)
    if normalize:
        cm = cm.astype('float') / np.maximum(cm.sum(axis=1)[:, np.newaxis], np.finfo(np.float64).eps)
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.get_cmap('Blues'))
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]), xticklabels=categories, yticklabels=categories, ylabel='Annotation', xlabel='Prediction')
    # Rotate the tick labels and set their alignment
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    # Loop over data dimensions and create text annotations
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt), ha="center", va="center", color="white" if cm[i, j] > thresh else "black", fontsize=int(12-pow(len(categories), 0.5)))
    fig.tight_layout()
    plt.show(fig)


In [ ]:
labels_pred = my_model.predict(instances_dev)
cm = confusion_matrix(np.argmax(labels_dev, axis=1), np.argmax(labels_pred, axis=1))
draw_confusion_matrix(cm, ["Poor", "Intermediate", "Good", "Excellent"])

## Step 8: Save results

In [ ]:
RESULTS_FOLDER = "../Results"
RESULTS_FILE = f"{RESULTS_FOLDER}/models_performance.csv"

In [ ]:
results_df = pd.read_csv(RESULTS_FILE, index_col = 0)

In [ ]:
COLUMNS = [
    'model_name',
    'epochs',
    'learning_rate',
    'batch_size',
    'activation',
    'loss',
    'optimizer',
    'regularization',
    'initializer',
    'train_accuracy',
    'validation_accuracy',
    'bias',
    'variance',
    'training_time'
]

new_result = [[
    model_name,
    N_EPOCHS,
    LEARNING_RATE,
    BATCH_SIZE,
    ACTIVATION,
    "categorical_crossentropy",
    "SGD",
    "l1 0.0001",
    "he_normal",
    training_accuracy,
    val_accuracy,
    bias,
    variance,
    training_time]]

new_result_df = pd.DataFrame(new_result, columns=COLUMNS)
concatenation = pd.concat([results_df, new_result_df], ignore_index=True)
concatenation.to_csv(RESULTS_FILE)


In [ ]:
# last_five = results_df.tail(5)
last_five = concatenation.tail(5)
print(last_five)
print("\n---Metrics---")
print(last_five[["train_accuracy", "validation_accuracy", "bias", "variance", "training_time"]].describe())

In [ ]:
MODELS_FOLDER = "../Models"
save_name = f"{MODELS_FOLDER}/{model_name}-4.h5"
my_model.save(save_name)